# Trabajo práctico integrador - Análisis de datos

## Limpieza y preparación de datos / Ingeniería de features

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, KBinsDiscretizer, PowerTransformer
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV

import pickle

In [2]:
# Set color palette
color = ['#1ED760', '#FAF5F5']
sns.set_palette(sns.color_palette(color))

In [3]:
# Load dataset from GitHub repository
df_original = pd.read_csv('data/data_playlist.csv')
df = df_original.copy(deep=True)

In [4]:
# Show first 10 rows
df.head(10)

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,label
0,0.713,0.514,100125,0.521,0.816000,8,0.1120,-14.835,0,0.0444,119.879,4,0.143,1
1,0.192,0.714,207019,0.614,0.000000,4,0.2630,-6.935,1,0.0319,123.969,4,0.582,1
2,0.333,0.630,216200,0.455,0.000004,5,0.1270,-9.290,1,0.0292,139.931,4,0.199,1
3,0.601,0.810,136413,0.221,0.210000,5,0.1840,-11.005,1,0.0429,109.960,4,0.798,1
4,0.883,0.465,181440,0.459,0.000173,6,0.0692,-8.137,0,0.0351,90.807,4,0.288,1
5,0.524,0.633,244360,0.401,0.000000,4,0.1230,-12.549,1,0.0439,134.978,4,0.523,1
6,0.597,0.507,183573,0.795,0.000000,9,0.2960,-6.966,1,0.0607,165.540,4,0.900,0
7,0.452,0.825,259102,0.435,0.609000,1,0.0953,-9.582,1,0.0568,119.038,4,0.243,1
8,0.748,0.420,366179,0.324,0.839000,9,0.0723,-14.700,0,0.0556,183.020,3,0.330,1
9,0.913,0.292,197613,0.246,0.088300,0,0.2090,-9.758,1,0.0330,140.316,4,0.249,1


In [5]:
# Show dataset shape
df.shape

(750, 14)

## 1. Análisis de datos faltantes

In [6]:
df.isna().sum()

acousticness        0
danceability        0
duration            0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
label               0
dtype: int64

- No se evidencia la presencia de valores faltantes ni nulos, por este motivo, no se aplican técnicas de imputación.

## 2. Pre-procesamiento de las variables

In [7]:
numerical_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']
categorical_features = ['key', 'mode', 'time_signature']
special_categorical_features = ['tempo']

In [8]:
X = df[numerical_features + categorical_features + special_categorical_features]
y = df['label']

In [9]:
def print_info(var, is_numeric=True):
    print("*" * 50)
    print(f"Variable: {var}")
    cant_unique = len(df[var].unique())
    if is_numeric:
        print(f"La variable oscila entre los siguientes valores: {df[var].min()} - {df[var].max()}")
    else:
        
        if cant_unique < 10:
            print(f"Categorías: {df[var].unique()}")
    print(f"La variable tiene {cant_unique} valores únicos")

### Definición de las técnicas a utilizar:

### 1. Variables numéricas:

### Información

In [10]:
for var in numerical_features:
    print_info(var)

**************************************************
Variable: acousticness
La variable oscila entre los siguientes valores: 1.17e-06 - 0.994
La variable tiene 596 valores únicos
**************************************************
Variable: danceability
La variable oscila entre los siguientes valores: 0.107 - 0.986
La variable tiene 458 valores únicos
**************************************************
Variable: energy
La variable oscila entre los siguientes valores: 0.00925 - 0.995
La variable tiene 502 valores únicos
**************************************************
Variable: instrumentalness
La variable oscila entre los siguientes valores: 0.0 - 0.967
La variable tiene 431 valores únicos
**************************************************
Variable: liveness
La variable oscila entre los siguientes valores: 0.024 - 0.979
La variable tiene 445 valores únicos
**************************************************
Variable: loudness
La variable oscila entre los siguientes valores: -29.601 - -0.5

### Técnicas a utilizar

- **Feature Scaling**: Estandarización, Escalado a mínimo-máximo
- **Transformación de variables**: Yeo-Johnson (debido a que admite variables positivas y negativas)

### 2. Variables categóricas

### Información

In [11]:
for var in categorical_features:
    print_info(var, is_numeric=False)

**************************************************
Variable: key
La variable tiene 12 valores únicos
**************************************************
Variable: mode
Categorías: [0 1]
La variable tiene 2 valores únicos
**************************************************
Variable: time_signature
Categorías: [4 3 5 1]
La variable tiene 4 valores únicos


In [12]:
print_info('tempo')

**************************************************
Variable: tempo
La variable oscila entre los siguientes valores: 55.747 - 204.162
La variable tiene 729 valores únicos


### Técnicas a utilizar

**Variables**:
- key
- mode
- tempo

Codificación: One-Hot-Encoding

**Variable**:
time_Signature

Codificación: Discretización (binning) + One-Hot-Encoding

## Pipelines de pre-procesamiento

#### Pipelines variables numéricas

In [13]:
numeric_std_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson'))])

In [14]:
numeric_minmax_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('power', PowerTransformer(method='yeo-johnson'))])

#### Pipelines variables categóricas

In [15]:
categorical_binning = KBinsDiscretizer(n_bins=10, encode='onehot' , strategy='uniform' )

In [16]:
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

### Column transformer

In [17]:
preprocessor_std = ColumnTransformer(
    transformers=[
        ('num', numeric_std_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('cat_bin', categorical_binning, special_categorical_features)])

In [18]:
preprocessor_min_max = ColumnTransformer(
    transformers=[
        ('num', numeric_minmax_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('cat_bin', categorical_binning, special_categorical_features)])

### Pipeline 

In [19]:
preprocessing_std = Pipeline(steps=[('preprocessor', preprocessor_std),])

In [20]:
preprocessing_min_max = Pipeline(steps=[('preprocessor', preprocessor_min_max),])

### Experimentation

#### Define models

In [21]:
logistic_regresion_std_model = Pipeline(steps=[('preprocessor', preprocessing_std),
                       ('classifier', LogisticRegression())])

In [22]:
logistic_regresion_min_max_model = Pipeline(steps=[('preprocessor', preprocessing_min_max),
                       ('classifier', LogisticRegression())])

In [23]:
decision_tree_std_pipe = Pipeline(steps=[('preprocessor', preprocessing_std),
                       ('classifier', DecisionTreeClassifier())])

In [24]:
decision_tree_min_max_pipe = Pipeline(steps=[('preprocessor', preprocessing_min_max),
                       ('classifier', DecisionTreeClassifier())])

In [25]:
random_forest_std_pipe = Pipeline(steps=[('preprocessor', preprocessing_std),
                       ('classifier', RandomForestClassifier())])

In [26]:
random_forest_min_max_pipe = Pipeline(steps=[('preprocessor', preprocessing_min_max),
                       ('classifier', RandomForestClassifier())])

In [27]:
xgb_std = Pipeline(steps=[('preprocessor', preprocessing_std),
                       ('classifier', xgb.XGBClassifier())])

In [28]:
xgb_min_max = Pipeline(steps=[('preprocessor', preprocessing_min_max),
                       ('classifier', xgb.XGBClassifier())])

In [29]:
param_grid = {
    'classifier__max_depth': [15, 30, 45],
    'classifier__criterion': ['gini', 'entropy', 'log_loss'],
}

In [30]:
random_forest_params = {
    'classifier__n_estimators': [50, 100, 150],
}

In [31]:
param_grid_random_forest = dict(param_grid)
param_grid_random_forest.update(random_forest_params)

In [32]:
# Grid Search - Models
gs_decision_tree_std = GridSearchCV(decision_tree_std_pipe, param_grid, cv=5, n_jobs=-1, verbose=3)
gs_decision_tree_min_max = GridSearchCV(decision_tree_min_max_pipe, param_grid, cv=5, n_jobs=-1, verbose=3)

gs_random_forest_std = GridSearchCV(random_forest_std_pipe, param_grid, cv=5, n_jobs=-1, verbose=3)
gs_random_forest_min_max = GridSearchCV(random_forest_min_max_pipe, param_grid, cv=5, n_jobs=-1, verbose=3)

gs_xgb_std = GridSearchCV(xgb_std, param_grid, cv=5, n_jobs=-1, verbose=3)
gs_xgb_min_max = GridSearchCV(xgb_min_max, param_grid, cv=5, n_jobs=-1, verbose=3)

#### Metric report

In [33]:
def metric_report(y_test, y_pred, y_proba):  
    print(classification_report(y_test, y_pred))  
    print('Area bajo la curva ROC:',np.round(roc_auc_score(y_test, y_proba[:,1]), 4)) 
    precision, recall,threshold=precision_recall_curve(y_test, y_proba[:,1]);
    print('Area bajo la curva Precision-Recall:',np.round(auc(recall, precision), 4))

#### Train-Test Split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Experimentation

#### 1. Logistic Regression - Numeric: Standard Scaler + Yeo-Jhonson

In [35]:
logistic_regresion_std_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier', LogisticRegression())])

In [36]:
y_pred = logistic_regresion_std_model.predict(X_test)
y_proba = logistic_regresion_std_model.predict_proba(X_test)

In [37]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.67      0.76      0.71        54
           1       0.85      0.79      0.82        96

    accuracy                           0.78       150
   macro avg       0.76      0.78      0.77       150
weighted avg       0.79      0.78      0.78       150

Area bajo la curva ROC: 0.8773
Area bajo la curva Precision-Recall: 0.9142


#### 2. Logistic Regression - Numeric: MinMax + Yeo-Jhonson 

In [38]:
logistic_regresion_min_max_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier', LogisticRegression())])

In [39]:
y_pred = logistic_regresion_std_model.predict(X_test)
y_proba = logistic_regresion_std_model.predict_proba(X_test)

In [40]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.67      0.76      0.71        54
           1       0.85      0.79      0.82        96

    accuracy                           0.78       150
   macro avg       0.76      0.78      0.77       150
weighted avg       0.79      0.78      0.78       150

Area bajo la curva ROC: 0.8773
Area bajo la curva Precision-Recall: 0.9142


### Decision Tree

#### Decision tree: Numeric: Standard Scaler

In [41]:
gs_decision_tree_std.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           StandardScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                                          ['key',
                                                                                           'mode',
                                                                                           'time_signature']),
                                                                                         ('cat_bin',
                                                                                          KBinsDiscretizer(n_bins=10,
                                                                                                           strategy='uniform'),
                                                                                          ['tempo'])]))])),
                                       ('classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [42]:
pd.DataFrame(gs_decision_tree_std.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.084377,0.007654,0.012499,0.006250,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.808333,0.816667,0.716667,0.741667,0.700000,0.756667,0.047551,9
1,0.071880,0.007649,0.015626,0.000012,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.825000,0.808333,0.750000,0.775000,0.733333,0.778333,0.034400,6
2,0.074999,0.006249,0.015625,0.009883,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.833333,0.800000,0.733333,0.758333,0.716667,0.768333,0.042947,8
3,0.068752,0.007652,0.015627,0.000009,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.800000,0.783333,0.725000,0.800000,0.750000,0.771667,0.029627,7
4,0.068750,0.007660,0.018751,0.006254,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.791667,0.783333,0.733333,0.825000,0.766667,0.780000,0.030092,5
5,0.068747,0.007658,0.009375,0.007655,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.800000,0.808333,0.741667,0.791667,0.758333,0.780000,0.025604,4
6,0.068747,0.007656,0.012502,0.006251,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.808333,0.808333,0.750000,0.808333,0.783333,0.791667,0.022973,1
7,0.075003,0.006251,0.015624,0.000005,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.816667,0.783333,0.725000,0.816667,0.766667,0.781667,0.034319,3
8,0.071873,0.007653,0.012499,0.006250,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.808333,0.783333,0.750000,0.825000,0.741667,0.781667,0.032232,2


In [43]:
gs_decision_tree_std.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier',
                 DecisionTreeClassifier(criterion='log_loss', max_depth=15))])

In [44]:
gs_decision_tree_std.best_score_

0.7916666666666666

In [45]:
y_pred = gs_decision_tree_std.predict(X_test)
y_proba = gs_decision_tree_std.predict_proba(X_test)

In [46]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.72      0.81      0.77        54
           1       0.89      0.82      0.85        96

    accuracy                           0.82       150
   macro avg       0.80      0.82      0.81       150
weighted avg       0.83      0.82      0.82       150

Area bajo la curva ROC: 0.8189
Area bajo la curva Precision-Recall: 0.9119


#### Decision tree: Numeric: Min - Max

In [47]:
gs_decision_tree_min_max.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           MinMaxScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                                          ['key',
                                                                                           'mode',
                                                                                           'time_signature']),
                                                                                         ('cat_bin',
                                                                                          KBinsDiscretizer(n_bins=10,
                                                                                                           strategy='uniform'),
                                                                                          ['tempo'])]))])),
                                       ('classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [48]:
pd.DataFrame(gs_decision_tree_min_max.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.097413,0.003959,0.013828,0.002464,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.816667,0.800000,0.758333,0.775000,0.708333,0.771667,0.037491,8
1,0.078876,0.014720,0.010796,0.006121,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.808333,0.833333,0.775000,0.791667,0.750000,0.791667,0.028382,1
2,0.065627,0.006251,0.015625,0.000002,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.816667,0.808333,0.750000,0.733333,0.725000,0.766667,0.038370,9
3,0.071874,0.012501,0.018752,0.006251,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.791667,0.783333,0.725000,0.816667,0.750000,0.773333,0.032232,7
4,0.071875,0.007656,0.012500,0.006250,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.775000,0.800000,0.741667,0.791667,0.783333,0.778333,0.020138,5
5,0.076220,0.010413,0.012501,0.006251,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.808333,0.766667,0.741667,0.825000,0.775000,0.783333,0.029814,4
6,0.074974,0.006236,0.013137,0.004977,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.816667,0.791667,0.766667,0.808333,0.766667,0.790000,0.020683,2
7,0.062498,0.000003,0.017935,0.004616,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.816667,0.800000,0.716667,0.825000,0.783333,0.788333,0.038586,3
8,0.061069,0.011533,0.012499,0.006250,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.808333,0.800000,0.716667,0.800000,0.750000,0.775000,0.035746,6


In [49]:
gs_decision_tree_min_max.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier', DecisionTreeClassifier(max_depth=30))])

In [50]:
gs_decision_tree_min_max.best_score_

0.7916666666666666

In [51]:
y_pred = gs_decision_tree_min_max.predict(X_test)
y_proba = gs_decision_tree_min_max.predict_proba(X_test)

In [52]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.70      0.74      0.72        54
           1       0.85      0.82      0.84        96

    accuracy                           0.79       150
   macro avg       0.78      0.78      0.78       150
weighted avg       0.80      0.79      0.79       150

Area bajo la curva ROC: 0.7818
Area bajo la curva Precision-Recall: 0.8929


In [53]:
# save the model
filename = 'models/decision_tree_min_max.sav'
pickle.dump(gs_decision_tree_min_max, open(filename, 'wb'))

### Random Forest

#### Random Forest - Numeric: Standard Scaler

In [54]:
gs_random_forest_std.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           StandardScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                                          ['key',
                                                                                           'mode',
                                                                                           'time_signature']),
                                                                                         ('cat_bin',
                                                                                          KBinsDiscretizer(n_bins=10,
                                                                                                           strategy='uniform'),
                                                                                          ['tempo'])]))])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [55]:
pd.DataFrame(gs_random_forest_std.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.428850,0.021300,0.049999,1.530989e-02,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.783333,0.875000,0.783333,0.866667,0.800000,0.821667,0.040689,4
1,0.392555,0.013068,0.042547,9.388787e-03,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.808333,0.866667,0.791667,0.866667,0.783333,0.823333,0.036286,3
2,0.379168,0.007605,0.035377,6.068684e-03,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.791667,0.858333,0.766667,0.866667,0.791667,0.815000,0.039930,7
3,0.414725,0.011876,0.034222,6.332478e-03,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.791667,0.866667,0.791667,0.850000,0.791667,0.818333,0.033082,6
4,0.410013,0.006757,0.034375,6.250859e-03,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.783333,0.850000,0.758333,0.850000,0.800000,0.808333,0.036515,9
5,0.402986,0.012316,0.034377,6.249833e-03,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.800000,0.883333,0.800000,0.850000,0.800000,0.826667,0.034319,1
6,0.412499,0.007656,0.031251,7.008046e-07,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.800000,0.891667,0.783333,0.866667,0.783333,0.825000,0.045338,2
7,0.400000,0.012499,0.034376,6.250167e-03,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.800000,0.866667,0.766667,0.875000,0.758333,0.813333,0.049046,8
8,0.375004,0.048398,0.031253,9.880082e-03,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.808333,0.866667,0.791667,0.866667,0.775000,0.821667,0.038224,4


In [56]:
gs_random_forest_std.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', max_depth=45))])

In [57]:
gs_random_forest_std.best_score_

0.8266666666666668

In [58]:
y_pred = gs_random_forest_std.predict(X_test)
y_proba = gs_random_forest_std.predict_proba(X_test)

In [59]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.74      0.83      0.78        54
           1       0.90      0.83      0.86        96

    accuracy                           0.83       150
   macro avg       0.82      0.83      0.82       150
weighted avg       0.84      0.83      0.84       150

Area bajo la curva ROC: 0.9133
Area bajo la curva Precision-Recall: 0.9436


#### Random Forest - Numeric: Min Max Scaler


In [60]:
gs_random_forest_min_max.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           MinMaxScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                                          ['key',
                                                                                           'mode',
                                                                                           'time_signature']),
                                                                                         ('cat_bin',
                                                                                          KBinsDiscretizer(n_bins=10,
                                                                                                           strategy='uniform'),
                                                                                          ['tempo'])]))])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [61]:
pd.DataFrame(gs_random_forest_min_max.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.359585,0.012194,0.031249,0.000003,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.775000,0.891667,0.783333,0.858333,0.791667,0.820000,0.046428,4
1,0.364182,0.005726,0.031248,0.000004,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.808333,0.858333,0.750000,0.875000,0.766667,0.811667,0.049046,8
2,0.384603,0.036731,0.034376,0.006255,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.800000,0.858333,0.741667,0.866667,0.758333,0.805000,0.050717,9
3,0.425809,0.024516,0.040624,0.007651,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.791667,0.866667,0.766667,0.875000,0.791667,0.818333,0.043906,6
4,0.403126,0.006251,0.031251,0.000002,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.800000,0.883333,0.783333,0.833333,0.808333,0.821667,0.034801,3
5,0.393750,0.011692,0.031251,0.000001,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.808333,0.883333,0.783333,0.891667,0.775000,0.828333,0.049610,1
6,0.402993,0.006190,0.032811,0.003121,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.825000,0.841667,0.775000,0.858333,0.775000,0.815000,0.034319,7
7,0.403299,0.011194,0.031251,0.000003,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.775000,0.841667,0.791667,0.883333,0.808333,0.820000,0.038586,5
8,0.375006,0.055009,0.028121,0.006262,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.783333,0.900000,0.783333,0.883333,0.775000,0.825000,0.054772,2


In [62]:
gs_random_forest_min_max.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_signature']),
                                                                  ('cat_bin',
                                                                   KBinsDiscretizer(n_bins=10,
                                                                                    strategy='uniform'),
                                                                   ['tempo'])]))])),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', max_depth=45))])

In [63]:
gs_random_forest_min_max.best_score_

0.8283333333333334

In [64]:
y_pred = gs_random_forest_min_max.predict(X_test)
y_proba = gs_random_forest_min_max.predict_proba(X_test)

In [65]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.78      0.83      0.80        54
           1       0.90      0.86      0.88        96

    accuracy                           0.85       150
   macro avg       0.84      0.85      0.84       150
weighted avg       0.86      0.85      0.85       150

Area bajo la curva ROC: 0.9041
Area bajo la curva Precision-Recall: 0.9331


In [66]:
# save the model
filename = 'models/random_forest_min_max.sav'
pickle.dump(gs_random_forest_min_max, open(filename, 'wb'))

#### XGBoost - Numeric: Standard Scaler

In [67]:
gs_xgb_std.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[18:29:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           StandardScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='...
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None,
                                                      reg_alpha=None,
                                                      reg_lambda=None, ...))]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [68]:
pd.DataFrame(gs_xgb_std.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.390702,0.013884,0.015623,0.000005,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
1,0.390861,0.009886,0.015626,0.000002,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
2,0.391213,0.009579,0.015626,0.000002,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
3,0.393947,0.011908,0.018750,0.006249,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
4,0.393750,0.011693,0.015624,0.000002,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
5,0.393749,0.015309,0.021877,0.007655,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
6,0.390626,0.013977,0.018751,0.006249,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
7,0.419570,0.015359,0.018931,0.002382,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
8,0.423619,0.082703,0.013959,0.007941,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1


In [69]:
gs_xgb_std.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time...
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=15, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0, ...))])

In [70]:
gs_xgb_std.best_score_

0.8133333333333332

In [71]:
y_pred = gs_xgb_std.predict(X_test)
y_proba = gs_xgb_std.predict_proba(X_test)

In [72]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.75      0.81      0.78        54
           1       0.89      0.84      0.87        96

    accuracy                           0.83       150
   macro avg       0.82      0.83      0.82       150
weighted avg       0.84      0.83      0.83       150

Area bajo la curva ROC: 0.8872
Area bajo la curva Precision-Recall: 0.8986


#### XGBoost - Numeric: Min Max Scaler

In [73]:
gs_xgb_min_max.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[18:29:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('scaler',
                                                                                                           MinMaxScaler()),
                                                                                                          ('power',
                                                                                                           PowerTransformer())]),
                                                                                          ['acousticness',
                                                                                           'danceability',
                                                                                           'energy',
                                                                                           'instrumentalness',
                                                                                           'liveness',
                                                                                           'loudness',
                                                                                           'speechiness',
                                                                                           'valence']),
                                                                                         ('cat',
                                                                                          OneHotEncoder(handle_unknown='ig...
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None,
                                                      reg_alpha=None,
                                                      reg_lambda=None, ...))]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy',
                                                   'log_loss'],
                         'classifier__max_depth': [15, 30, 45]},
             verbose=3)

In [74]:
pd.DataFrame(gs_xgb_min_max.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__criterion,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.543352,0.044262,0.024923,0.004954,gini,15,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
1,0.493853,0.028942,0.022201,0.002313,gini,30,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
2,0.486012,0.023678,0.022075,0.002126,gini,45,"{'classifier__criterion': 'gini', 'classifier_...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
3,0.442084,0.028309,0.018902,0.006551,entropy,15,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
4,0.508003,0.090970,0.045763,0.014698,entropy,30,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
5,0.646772,0.016287,0.029126,0.010925,entropy,45,"{'classifier__criterion': 'entropy', 'classifi...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
6,0.603393,0.028507,0.031120,0.009861,log_loss,15,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
7,0.581579,0.038425,0.019344,0.008671,log_loss,30,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1
8,0.454878,0.023949,0.029759,0.010450,log_loss,45,"{'classifier__criterion': 'log_loss', 'classif...",0.825,0.816667,0.766667,0.866667,0.791667,0.813333,0.033582,1


In [75]:
gs_xgb_min_max.best_estimator_

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler()),
                                                                                   ('power',
                                                                                    PowerTransformer())]),
                                                                   ['acousticness',
                                                                    'danceability',
                                                                    'energy',
                                                                    'instrumentalness',
                                                                    'liveness',
                                                                    'loudness',
                                                                    'speechiness',
                                                                    'valence']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['key',
                                                                    'mode',
                                                                    'time_s...
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=15, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0, ...))])

In [76]:
gs_xgb_min_max.best_score_

0.8133333333333332

In [77]:
y_pred = gs_xgb_min_max.predict(X_test)
y_proba = gs_xgb_min_max.predict_proba(X_test)

In [78]:
metric_report(y_test, y_pred, y_proba)

              precision    recall  f1-score   support

           0       0.75      0.81      0.78        54
           1       0.89      0.84      0.87        96

    accuracy                           0.83       150
   macro avg       0.82      0.83      0.82       150
weighted avg       0.84      0.83      0.83       150

Area bajo la curva ROC: 0.8872
Area bajo la curva Precision-Recall: 0.8986


### Resultados:

Mejor modelo: Random Forest, con preprocesado numérico: Min-Max Scaler + Yeo Johnson. Intuimos que esto se debe a que la mayoría de nuestras variables numéricas se encuentran en el rango 0 - 1, por lo que al hacer min-max scaler, se mantiene la distribución original de los datos, la cual es "normalizada", empleando Yeo Johnson. De esta forma los datos resultan útiles para el modelo, motivo por el cual las métricas arrojadas por el mismo son alentadoras (F1 score 0.81 para la clase minoritaria y 0.89 para la mayoritaria).